# Elite100 Visualizer
This notebook loads `leaderboard_expanded_full-CLEANED.csv`, parses the `Time` column, and shows an interactive Brand vs Time scatter (each dot is one entry).

- X-axis: `Brand` (categorical)
- Y-axis: Time (seconds) — lower times are lower on the axis (fastest at the bottom)
- Hover shows `Name`, `Model`, `Rank`, and the original `Time`.

In [371]:
# Imports and configuration
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re
from pathlib import Path

DATA_PATH = Path('elite100.csv')
df = pd.read_csv(DATA_PATH, skipinitialspace=True)

# Strip spaces from column names
df.columns = df.columns.str.strip()

# Strip trailing spaces from string columns
string_cols = df.select_dtypes(include='object').columns
for col in string_cols:
    df[col] = df[col].str.strip()

# Quick peek
df.head()

,Rank,Name,Time,TrackWeapon,Brand,Model,Year,ChassisCode,EngineL,EngineType,Drivetrain,RaceEvent,Date,Comment
0,1,Paolo Mantolino,01:39.289,2024 Porsche 911 GT3,Porsche,911 GT3,2024,992.1,4.0,NA,RWD,Full Throttle Weekend (ETCC),August 17 2025,Set EngineType to 'NA' based on engine spec (4...
1,2,Angelo Magatoto,01:40.532,2020 Toyota Supra,Toyota,Supra,2020,A90,3.0,FI,RWD,Full Throttle Weekend (Time Battle),February 25 2024,NaN
2,3,Mark Savino,01:41.292,2019 Porsche 911 GT3 RS,Porsche,911 GT3 RS,2019,991.2,4.0,NA,RWD,Full Throttle Weekend (ETCC),August 17 2025,Set EngineType to 'NA' based on engine spec (4...
3,4,Thmoon Lee,01:41.547,2019 Lamborghini Huracan Performante,Lamborghini,Huracan Performante,2019,LP640,5.2,NA,AWD,Full Throttle Weekend (ETCC),July 21 2022,Set EngineType to 'NA' based on engine spec (5...
4,5,Pep Lo,01:42.190,2017 Nissan R35,Nissan,GT-R,2017,R35,3.8,FI,AWD,Full Throttle Weekend (Time Battle),May 11 2025,NaN


In [372]:
# Robust parser for time strings like '01:39.289' -> seconds (float)
def parse_time_to_seconds(t):
    if pd.isna(t):
        return np.nan
    s = str(t).strip()
    # Match mm:ss.sss or m:ss or ss.sss
    m = re.match(r'^(?:(\d+):)?(\d{1,2}(?:\.\d+)?)$', s)
    if m:
        min_part = m.group(1)
        sec_part = m.group(2)
        minutes = int(min_part) if min_part is not None else 0
        try:
            seconds = float(sec_part)
        except ValueError:
            return np.nan
        return minutes * 60.0 + seconds
    # If not matched, try to extract numbers and guess
    nums = re.findall(r'\d+\.?\d*', s)
    if len(nums) == 1:
        return float(nums[0])
    return np.nan

def seconds_to_duration(sec, decimals=3):
    """Convert seconds (float) to mm:ss format (decimals: 1-3)"""
    if pd.isna(sec):
        return ''
    minutes = int(sec // 60)
    remaining_sec = sec % 60
    if decimals == 1:
        return f'{minutes:02d}:{remaining_sec:04.1f}'
    else:  # default 3 decimals
        return f'{minutes:02d}:{remaining_sec:06.3f}'

# Add parsed column
df['Time_sec'] = df['Time'].apply(parse_time_to_seconds)
# Keep original `Time` for hover
df['Time_orig'] = df['Time']
# Create duration format for Y-axis display (mm:ss.sss)
df['Time_duration'] = df['Time_sec'].apply(seconds_to_duration)

# Drop rows with missing Brand or unparseable times
df_plot = df.dropna(subset=['Brand', 'Time_sec']).copy()
# Ensure Brand is treated as categorical (not numeric)
df_plot['Brand'] = df_plot['Brand'].astype(str)
# Clean engine displacement column if needed
df_plot['EngineL_clean'] = pd.to_numeric(df_plot['EngineL'], errors='coerce')

# Show parsed results sample with Brand info
fastest_time = df_plot['Time_sec'].min()
print(f'Total rows after filtering: {len(df_plot)}')
print(f'Unique brands: {df_plot["Brand"].nunique()}')
print(f'Fastest lap time: {seconds_to_duration(fastest_time)}')
print(f'\nBrand distribution:')
print(df_plot['Brand'].value_counts())
print(f'\nSample data:')
df_plot[['Rank','Name','Brand','Model','Time_orig','Time_sec','Time_duration']].head(10)

Total rows after filtering: 100
Unique brands: 18
Fastest lap time: 01:39.289

Brand distribution:
Brand
Honda            23
Porsche          18
Toyota           11
Subaru           10
BMW               8
Ford              7
Nissan            4
Volkswagen        4
Lotus             3
Audi              2
Mitsubishi        2
Chevrolet         2
Lamborghini       1
Mercedes-Benz     1
Aston Martin      1
Lexus             1
Dodge             1
Mazda             1
Name: count, dtype: int64

Sample data:


,Rank,Name,Brand,Model,Time_orig,Time_sec,Time_duration
0,1,Paolo Mantolino,Porsche,911 GT3,01:39.289,99.289,01:39.289
1,2,Angelo Magatoto,Toyota,Supra,01:40.532,100.532,01:40.532
2,3,Mark Savino,Porsche,911 GT3 RS,01:41.292,101.292,01:41.292
3,4,Thmoon Lee,Lamborghini,Huracan Performante,01:41.547,101.547,01:41.547
4,5,Pep Lo,Nissan,GT-R,01:42.190,102.190,01:42.190
5,6,Thomas Raldorf,Subaru,WRX STI,01:43.265,103.265,01:43.265
6,7,Richard Dy,Porsche,911 GT3 RS,01:43.812,103.812,01:43.812
7,8,Christian Ho,Ford,Shelby GT500,01:44.033,104.033,01:44.033
8,9,Rafael Lao,Nissan,GT-R Black Edition,01:44.441,104.441,01:44.441
9,10,Jonathan Padilla,Nissan,GT-R,01:44.833,104.833,01:44.833


In [381]:
# Simplified text-only visualization
# Sort brands by fastest time
brand_fastest_time = df_plot.groupby('Brand')['Time_sec'].min().sort_values()
brand_order_simple = brand_fastest_time.index.tolist()
brand_position_simple = {brand: i for i, brand in enumerate(brand_order_simple)}
df_plot['Brand_X_simple'] = df_plot['Brand'].map(brand_position_simple)

# Calculate max text width needed per brand (approximate character count)
brand_max_text_width = {}
for brand in brand_order_simple:
    df_brand = df_plot[df_plot['Brand'] == brand]
    max_length = 0
    for idx, row in df_brand.iterrows():
        label_text = f"{row['Model']} {row['ChassisCode']}"
        max_length = max(max_length, len(label_text))
    brand_max_text_width[brand] = max_length

# Estimate character width (roughly 0.08 units per character at default zoom)
char_width = 0.08
max_brand_width = max(brand_max_text_width.values()) * char_width

# Create new figure with text only
fig_simple = go.Figure()

# Add vertical lines for each brand (dark theme)
for i in range(len(brand_order_simple)):
    fig_simple.add_vline(x=i, line_dash='solid', line_color='rgba(0,0,0,0.5)', layer='below')

# Track placed text items per brand for collision detection
placed_by_brand = {}
for brand in brand_order_simple:
    placed_by_brand[brand] = []

# Sort all entries by brand and time to maintain vertical order
df_sorted = df_plot.sort_values(['Brand_X_simple', 'Time_sec']).reset_index(drop=True)

# Get min and max times for later reference
min_time = df_plot['Time_sec'].min()

# Define legend order
legend_order = [
    ('NA', 'RWD'), ('FI', 'RWD'),
    ('NA', 'AWD'), ('FI', 'AWD'),
    ('NA', 'FWD'), ('FI', 'FWD')
]

# First pass: add traces in legend order for proper legend appearance
for engine_type, drivetrain in legend_order:
    df_subset = df_sorted[(df_sorted['EngineType'] == engine_type) & (df_sorted['Drivetrain'] == drivetrain)]
    
    if len(df_subset) == 0:
        continue
    
    legend_name = f"{drivetrain} {engine_type}"
    is_first = True
    
    for idx, row in df_subset.iterrows():
        y_pos = row['Time_sec']
        x_pos = row['Brand_X_simple']
        brand = row['Brand']
        
        hover_text = (
            f"<b>{row['Name']}</b><br>"
            f"Model: {row['Model']}<br>"
            f"Brand: {row['Brand']}<br>"
            f"Time: {row['Time_duration']}<br>"
            f"Engine: {row['EngineL']}L ({row['EngineType']})<br>"
            f"Drivetrain: {row['Drivetrain']}<br>"
            f"Rank: {row['Rank']}"
        )
        
        # Get drivetrain color
        drivetrain_colors_simple = {
            'RWD': 'rgb(255,0,0)',       # Red
            'AWD': 'rgb(0,100,255)',    # Blue
            'FWD': 'rgb(0,255,0)'       # Green
        }
        text_color = drivetrain_colors_simple[drivetrain]
        
        # Determine marker style based on engine type
        marker_line = dict(width=2, color=text_color) if engine_type == 'FI' else dict(width=0)
        
        # Create label text
        label_text = f"{row['Model']} {row['ChassisCode']}"
        
        # Add dot marker on the vertical line
        fig_simple.add_trace(go.Scatter(
            x=[x_pos],
            y=[y_pos],
            mode='markers',
            marker=dict(size=6, color=text_color if engine_type == 'NA' else 'rgba(0,0,0,0)', 
                       line=marker_line, symbol='circle'),
            hovertemplate=hover_text + '<extra></extra>',
            name=legend_name,
            showlegend=is_first,
            legendgroup=legend_name
        ))
        
        # Position text at original location
        fig_simple.add_trace(go.Scatter(
            x=[x_pos + 0.10],
            y=[y_pos],
            mode='text',
            text=[label_text],
            textposition='middle right',
            textfont=dict(size=9, color='white'),
            hovertemplate=hover_text + '<extra></extra>',
            showlegend=False,
            legendgroup=legend_name
        ))
        
        is_first = False

# Configure axes
min_time = df_plot['Time_sec'].min()
max_time = df_plot['Time_sec'].max()

tick_interval = 0.5
tick_values = np.arange(np.ceil(min_time / tick_interval) * tick_interval, 
                        max_time + tick_interval, 
                        tick_interval)
# Add the exact minimum time as a tick point
tick_values = np.sort(np.unique(np.concatenate([[min_time], tick_values])))

# Only show labels for 1-second intervals and minimum time
tick_labels = []
for t in tick_values:
    if t == min_time or (t % 1.0 < 0.01):  # Show label if it's min_time or a whole second
        tick_labels.append(seconds_to_duration(t, decimals=3))
    else:
        tick_labels.append('')  # Empty label for 0.5 sec intervals

fig_simple.update_yaxes(
    title_text='Lap Time - lowest is fastest', 
    tickvals=tick_values, 
    ticktext=tick_labels,
    showgrid=True,
    gridwidth=1,
    gridcolor='rgb(80,80,80)'
)

# Set x-axis range to prevent text overlap: each brand column gets spacing for its max text width
x_max = len(brand_order_simple) - 1 + max_brand_width + 0.5
fig_simple.update_xaxes(
    title_text='Brand (sorted by fastest time →)',
    tickvals=list(range(len(brand_order_simple))), 
    ticktext=brand_order_simple, 
    tickangle=45,
    range=[-0.5, x_max]
)

fig_simple.update_layout(
    title='Elite 100 - Text Only View',
    height=1000, 
    width=int(2000 + max_brand_width * 200),  # Dynamic width based on text length
    margin=dict(l=60, r=30, t=80, b=160), 
    hovermode='closest',
    plot_bgcolor='rgb(50,50,50)',
    paper_bgcolor='rgb(0,0,0)',
    font=dict(color='white'),
    xaxis=dict(gridcolor='rgb(80,80,80)', zerolinecolor='rgb(80,80,80)'),
    yaxis=dict(gridcolor='rgb(80,80,80)', zerolinecolor='rgb(80,80,80)'),
    legend=dict(
        traceorder='normal',
        yanchor='top',
        y=0.99,
        xanchor='right',
        x=0.99
    )
)

print(f'Simplified text-only visualization created with {len(df_plot)} entries across {len(brand_order_simple)} brands')
print(f'Max text width per brand: {max_brand_width:.2f} units')

Simplified text-only visualization created with 100 entries across 18 brands
Max text width per brand: 2.40 units


In [374]:
# Collision detection function for text labels
def has_collision(x, y, width, height, placed_items, margin=0.1):
    """
    Check if a text box at (x, y) with given dimensions collides with any placed items.
    margin: extra space around text to prevent close overlaps
    """
    x_min = x
    x_max = x + width
    y_min = y - height / 2
    y_max = y + height / 2
    
    for px, py, pw, ph in placed_items:
        px_min = px
        px_max = px + pw
        py_min = py - ph / 2
        py_max = py + ph / 2
        
        # Check if boxes overlap (with margin)
        if not (x_max + margin < px_min or x_min - margin > px_max or 
                y_max + margin < py_min or y_min - margin > py_max):
            return True
    return False

def find_non_colliding_position(x, y, width, height, placed_items, max_offset=2.0, step=0.3):
    """
    Find a vertical offset for text to avoid collisions.
    Tries offsets up and down from original position.
    """
    if not has_collision(x, y, width, height, placed_items):
        return y
    
    # Try offsets alternating up and down
    for offset in [step * i for i in range(1, int(max_offset / step) + 1)]:
        # Try positive offset
        if not has_collision(x, y + offset, width, height, placed_items):
            return y + offset
        # Try negative offset
        if not has_collision(x, y - offset, width, height, placed_items):
            return y - offset
    
    # If no collision-free position found, return original
    return y


In [382]:
fig_simple

In [376]:
# Optional: save interactive plot to standalone HTML file
# Uncomment and run to export
fig.write_html('brand_vs_time.html')
print('Saved: brand_vs_time.html')

Saved: brand_vs_time.html
